In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
import os
import sys
import numpy as np
import pandas as pd
import pyprind 
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
import matplotlib.pyplot as plt
from sklearn.calibration import CalibratedClassifierCV

In [2]:
data = pd.read_csv('./dataset_for_prod2/training_data.csv')

In [3]:
data

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [5]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Grigory\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
text = data['text']
label = data['label']

In [ ]:
C = TfidfVectorizer(stop_words=stop_words, max_features=12000)

In [8]:
X = vectorizer.fit_transform(text)

In [9]:
y = data['label']

In [10]:
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df)

         00  000  000th   01   02   03   04   05   06   07  ...  ziggy  \
0       0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
1       0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
2       0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
3       0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
4       0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
...     ...  ...    ...  ...  ...  ...  ...  ...  ...  ...  ...    ...   
119995  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
119996  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
119997  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
119998  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
119999  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   

        zimbabwe  zimbabwean  zombie  zone  zones  zoo  zook  zurich  \
0            0.0         0.0     0.0   

In [11]:
frv = X[0]
nzi = frv.nonzero()[1]
words = vectorizer.get_feature_names_out()

for idx in nzi:
    print(f"{words[idx]}: {frv[0, idx]}")

green: 0.2236886447009313
seeing: 0.2824094746188896
ultra: 0.3032382743956441
band: 0.27279486842221146
dwindling: 0.33348138753215184
street: 0.19574832968106995
sellers: 0.32473856088531294
short: 0.21635098052538024
reuters: 0.23381561494659703
black: 0.22881926629457025
back: 0.1593486630325837
bears: 0.2557044094822201
st: 0.20512722820243465
wall: 0.40269808028915505


In [38]:
X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=0.3, random_state=42)

In [39]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [40]:
X_test = [str(x) for x in X_test]


In [41]:
X_test_tfidf = vectorizer.transform(X_test)

In [42]:
tree = DecisionTreeClassifier(random_state=42)
log_regression = LogisticRegression(max_iter=1000, random_state=42)
knn = KNeighborsClassifier()


In [43]:
print("Размер X_train_tfidf:", X_train_tfidf.shape)
print("Размер y_train:", len(y_train))

Размер X_train_tfidf: (84000, 12000)
Размер y_train: 84000


In [44]:
tree.fit(X_train_tfidf, y_train)


DecisionTreeClassifier(random_state=42)

In [45]:
log_regression.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [46]:
knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier()

In [47]:
tree_pred = tree.predict(X_test_tfidf)
lr_pred = log_regression.predict(X_test_tfidf)
knn_pred = knn.predict(X_test_tfidf)


In [50]:
print("Decision Tree Classifier:")
print("Accuracy:", accuracy_score(y_test, tree_pred))
print(classification_report(y_test, tree_pred))

Decision Tree Classifier:
Accuracy: 0.8026111111111112
              precision    recall  f1-score   support

           0       0.80      0.82      0.81      8932
           1       0.87      0.88      0.87      9096
           2       0.77      0.76      0.76      8943
           3       0.77      0.75      0.76      9029

    accuracy                           0.80     36000
   macro avg       0.80      0.80      0.80     36000
weighted avg       0.80      0.80      0.80     36000



In [51]:
print("\nLogistic Regression:")
print("Accuracy:", accuracy_score(y_test, lr_pred))
print(classification_report(y_test, lr_pred))


Logistic Regression:
Accuracy: 0.9136388888888889
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      8932
           1       0.95      0.98      0.97      9096
           2       0.88      0.89      0.88      8943
           3       0.89      0.89      0.89      9029

    accuracy                           0.91     36000
   macro avg       0.91      0.91      0.91     36000
weighted avg       0.91      0.91      0.91     36000



In [52]:
print("\nk-Nearest Neighbors:")
print("Accuracy:", accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred))


k-Nearest Neighbors:
Accuracy: 0.8945
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      8932
           1       0.93      0.96      0.95      9096
           2       0.86      0.87      0.86      8943
           3       0.89      0.86      0.87      9029

    accuracy                           0.89     36000
   macro avg       0.89      0.89      0.89     36000
weighted avg       0.89      0.89      0.89     36000

